In [1]:
!uv pip install trl peft -q --system --prerelease disallow

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from trl import SFTTrainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
import pandas as pd
import torch

# ✅ 1. Load + shuffle data
csv_path = "/content/train.csv"
df = pd.read_csv(csv_path)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Build prompt format
prompts = [
    f"### Question: {str(q).strip()}\n\n### Answer: {str(a).strip().lower().replace('%', '').replace(',', '')}"
    for q, a in zip(df["question"], df["answer"])
]

# ✅ 2. Tokenize
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 256

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

dataset = Dataset.from_dict({"text": prompts}).map(tokenize_fn)

# ✅ 3. Load model + LoRA
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto")

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, peft_config)

# ✅ 4. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# ✅ 5. Training args — minimal & working
training_args = TrainingArguments(
    output_dir="phi2-lora-working",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_steps=150,
    learning_rate=5e-5,
    warmup_steps=5,
    logging_steps=1,
    save_steps=50,
    save_total_limit=1,
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine_with_restarts",
    seed=42,
    report_to="none",
    fp16=torch.cuda.is_available(),
    bf16=False,
)

# ✅ 6. SFTTrainer (no eval, no metrics)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    data_collator=data_collator,
    formatting_func=None,
    args=training_args,
)

# ✅ 7. Train (🚀 no evals, just loss)
trainer.train()


FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [2]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━